# AlphaCare Insurance Solutions - Exploratory Data Analysis
## Task 1: Data Understanding and Initial Insights

**Objective**: Analyze South African car insurance data to identify low-risk segments and understand loss ratio patterns.

**Dataset Period**: February 2014 - August 2015

**Key Metric**: Loss Ratio = TotalClaims / TotalPremium

## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set visualization defaults
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Create output directory if it doesn't exist
import os
os.makedirs('../outputs/plots', exist_ok=True)

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
df = pd.read_csv('../data/raw/insurance.csv', low_memory=False)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

## 2. Initial Data Exploration

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Display last few rows
print("Last 5 rows of the dataset:")
df.tail()

In [ ]:
# Dataset info
print("Dataset Information:")
df.info()

In [ ]:
# Column names
print(f"Total columns: {len(df.columns)}\n")
print("Column names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

## 3. Data Type Conversions

In [ ]:
# Check current data types
print("Current data types:")
print(df.dtypes.value_counts())
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Convert date columns
date_columns = ['TransactionMonth', 'VehicleIntroDate']
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"Converted {col} to datetime")

# Extract temporal features from TransactionMonth
if 'TransactionMonth' in df.columns:
    df['TransactionYear'] = df['TransactionMonth'].dt.year
    df['TransactionMonthNum'] = df['TransactionMonth'].dt.month
    df['TransactionQuarter'] = df['TransactionMonth'].dt.quarter
    print("\nExtracted temporal features from TransactionMonth")

In [ ]:
# Convert categorical columns to category dtype for memory optimization
categorical_columns = [
    'Province', 'Gender', 'VehicleType', 'MaritalStatus', 'Title', 'Language',
    'CoverType', 'CoverCategory', 'Product', 'CoverGroup', 'Section',
    'make', 'Model', 'bodytype', 'Citizenship', 'LegalType', 'Bank', 'AccountType'
]

for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].astype('category')

print(f"Converted {len([c for c in categorical_columns if c in df.columns])} columns to category dtype")
print(f"\nNew memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 4. Missing Values Analysis

In [ ]:
# Calculate missing values
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100
})

missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values(
    'Missing_Percentage', ascending=False
).reset_index(drop=True)

print(f"Columns with missing values: {len(missing_data)}\n")
print(missing_data.to_string())

In [ ]:
# Visualize missing values for top columns
if len(missing_data) > 0:
    top_missing = missing_data.head(20)
    
    plt.figure(figsize=(12, 8))
    plt.barh(top_missing['Column'], top_missing['Missing_Percentage'], color='coral')
    plt.xlabel('Missing Percentage (%)')
    plt.title('Top 20 Columns with Missing Values', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig('../outputs/plots/missing_values.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Missing values plot saved to outputs/plots/missing_values.png")

## 5. Summary Statistics

In [ ]:
# Descriptive statistics for numeric columns
print("Summary Statistics for Numeric Columns:")
df.describe()

In [ ]:
# Key financial metrics
print("Key Financial Metrics:")
print("="*60)

if 'TotalPremium' in df.columns:
    print(f"\nTotal Premium:")
    print(f"  Sum: R {df['TotalPremium'].sum():,.2f}")
    print(f"  Mean: R {df['TotalPremium'].mean():,.2f}")
    print(f"  Median: R {df['TotalPremium'].median():,.2f}")
    print(f"  Std Dev: R {df['TotalPremium'].std():,.2f}")

if 'TotalClaims' in df.columns:
    print(f"\nTotal Claims:")
    print(f"  Sum: R {df['TotalClaims'].sum():,.2f}")
    print(f"  Mean: R {df['TotalClaims'].mean():,.2f}")
    print(f"  Median: R {df['TotalClaims'].median():,.2f}")
    print(f"  Std Dev: R {df['TotalClaims'].std():,.2f}")
    
    # Claim statistics
    policies_with_claims = (df['TotalClaims'] > 0).sum()
    claim_rate = (policies_with_claims / len(df)) * 100
    print(f"\nClaim Statistics:")
    print(f"  Policies with claims: {policies_with_claims:,} ({claim_rate:.2f}%)")
    print(f"  Policies without claims: {(df['TotalClaims'] == 0).sum():,}")

In [ ]:
# Categorical variable distributions
categorical_vars = ['Province', 'Gender', 'VehicleType', 'CoverType']

for var in categorical_vars:
    if var in df.columns:
        print(f"\n{var} Distribution:")
        print("="*60)
        value_counts = df[var].value_counts()
        percentages = (value_counts / len(df)) * 100
        
        dist_df = pd.DataFrame({
            'Count': value_counts,
            'Percentage': percentages
        })
        print(dist_df.to_string())

## 6. Loss Ratio Analysis

In [ ]:
# Calculate overall loss ratio
if 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    total_premium = df['TotalPremium'].sum()
    total_claims = df['TotalClaims'].sum()
    overall_loss_ratio = (total_claims / total_premium) * 100 if total_premium > 0 else 0
    
    print("Overall Loss Ratio Analysis")
    print("="*60)
    print(f"Total Premium Collected: R {total_premium:,.2f}")
    print(f"Total Claims Paid: R {total_claims:,.2f}")
    print(f"Overall Loss Ratio: {overall_loss_ratio:.2f}%")
    print(f"\nInterpretation: For every R1 in premium, R{overall_loss_ratio/100:.2f} is paid in claims")
    
    if overall_loss_ratio < 100:
        print(f"Status: PROFITABLE (Loss ratio < 100%)")
    else:
        print(f"Status: UNPROFITABLE (Loss ratio >= 100%)")

### 6.1 Loss Ratio by Province

In [ ]:
# Loss ratio by province
if 'Province' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    province_analysis = df.groupby('Province').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).reset_index()
    
    province_analysis.columns = ['Province', 'TotalPremium', 'TotalClaims', 'PolicyCount']
    province_analysis['LossRatio'] = (province_analysis['TotalClaims'] / province_analysis['TotalPremium']) * 100
    province_analysis = province_analysis.sort_values('LossRatio', ascending=True)
    
    print("\nLoss Ratio by Province:")
    print("="*80)
    print(province_analysis.to_string(index=False))
    
    # Visualize
    plt.figure(figsize=(12, 6))
    colors = ['green' if x < 100 else 'red' for x in province_analysis['LossRatio']]
    plt.barh(province_analysis['Province'], province_analysis['LossRatio'], color=colors, alpha=0.7)
    plt.axvline(x=100, color='black', linestyle='--', linewidth=2, label='Break-even (100%)')
    plt.xlabel('Loss Ratio (%)', fontsize=12)
    plt.ylabel('Province', fontsize=12)
    plt.title('Loss Ratio by Province\n(Green = Profitable, Red = Unprofitable)', 
              fontsize=14, fontweight='bold')
    plt.legend()
    plt.tight_layout()
    plt.savefig('../outputs/plots/loss_ratio_by_province.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\nPlot saved to outputs/plots/loss_ratio_by_province.png")

### 6.2 Loss Ratio by Vehicle Type

In [ ]:
# Loss ratio by vehicle type
if 'VehicleType' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    vehicle_analysis = df.groupby('VehicleType').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).reset_index()
    
    vehicle_analysis.columns = ['VehicleType', 'TotalPremium', 'TotalClaims', 'PolicyCount']
    vehicle_analysis['LossRatio'] = (vehicle_analysis['TotalClaims'] / vehicle_analysis['TotalPremium']) * 100
    vehicle_analysis = vehicle_analysis.sort_values('LossRatio', ascending=True)
    
    print("\nLoss Ratio by Vehicle Type:")
    print("="*80)
    print(vehicle_analysis.to_string(index=False))
    
    # Visualize top vehicle types
    top_vehicles = vehicle_analysis.nlargest(15, 'PolicyCount')
    
    plt.figure(figsize=(12, 8))
    colors = ['green' if x < 100 else 'red' for x in top_vehicles['LossRatio']]
    plt.barh(top_vehicles['VehicleType'], top_vehicles['LossRatio'], color=colors, alpha=0.7)
    plt.axvline(x=100, color='black', linestyle='--', linewidth=2, label='Break-even (100%)')
    plt.xlabel('Loss Ratio (%)', fontsize=12)
    plt.ylabel('Vehicle Type', fontsize=12)
    plt.title('Loss Ratio by Vehicle Type (Top 15 by Policy Count)\n(Green = Profitable, Red = Unprofitable)', 
              fontsize=14, fontweight='bold')
    plt.legend()
    plt.tight_layout()
    plt.savefig('../outputs/plots/loss_ratio_by_vehicle_type.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\nPlot saved to outputs/plots/loss_ratio_by_vehicle_type.png")

### 6.3 Loss Ratio by Gender

In [ ]:
# Loss ratio by gender
if 'Gender' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    gender_analysis = df.groupby('Gender').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).reset_index()
    
    gender_analysis.columns = ['Gender', 'TotalPremium', 'TotalClaims', 'PolicyCount']
    gender_analysis['LossRatio'] = (gender_analysis['TotalClaims'] / gender_analysis['TotalPremium']) * 100
    
    print("\nLoss Ratio by Gender:")
    print("="*80)
    print(gender_analysis.to_string(index=False))
    
    # Visualize
    plt.figure(figsize=(10, 6))
    colors = ['green' if x < 100 else 'red' for x in gender_analysis['LossRatio']]
    plt.bar(gender_analysis['Gender'], gender_analysis['LossRatio'], color=colors, alpha=0.7, edgecolor='black')
    plt.axhline(y=100, color='black', linestyle='--', linewidth=2, label='Break-even (100%)')
    plt.xlabel('Gender', fontsize=12)
    plt.ylabel('Loss Ratio (%)', fontsize=12)
    plt.title('Loss Ratio by Gender', fontsize=14, fontweight='bold')
    plt.legend()
    plt.tight_layout()
    plt.savefig('../outputs/plots/loss_ratio_by_gender.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\nPlot saved to outputs/plots/loss_ratio_by_gender.png")

## 7. Claim Frequency and Severity

In [ ]:
# Claim Frequency Analysis
if 'TotalClaims' in df.columns:
    print("Claim Frequency Analysis")
    print("="*60)
    
    # Overall claim frequency
    total_policies = len(df)
    policies_with_claims = (df['TotalClaims'] > 0).sum()
    claim_frequency = (policies_with_claims / total_policies) * 100
    
    print(f"Total Policies: {total_policies:,}")
    print(f"Policies with Claims: {policies_with_claims:,}")
    print(f"Overall Claim Frequency: {claim_frequency:.2f}%")
    
    # Claim frequency by province
    if 'Province' in df.columns:
        print("\nClaim Frequency by Province:")
        province_freq = df.groupby('Province').apply(
            lambda x: ((x['TotalClaims'] > 0).sum() / len(x)) * 100
        ).sort_values(ascending=False)
        print(province_freq.to_string())
    
    # Claim frequency by vehicle type
    if 'VehicleType' in df.columns:
        print("\nClaim Frequency by Vehicle Type (Top 10):")
        vehicle_freq = df.groupby('VehicleType').apply(
            lambda x: ((x['TotalClaims'] > 0).sum() / len(x)) * 100
        ).sort_values(ascending=False).head(10)
        print(vehicle_freq.to_string())

In [ ]:
# Claim Severity Analysis
if 'TotalClaims' in df.columns:
    print("\nClaim Severity Analysis")
    print("="*60)
    
    # Filter policies with claims
    df_with_claims = df[df['TotalClaims'] > 0].copy()
    
    if len(df_with_claims) > 0:
        avg_claim_severity = df_with_claims['TotalClaims'].mean()
        median_claim_severity = df_with_claims['TotalClaims'].median()
        
        print(f"Average Claim Severity: R {avg_claim_severity:,.2f}")
        print(f"Median Claim Severity: R {median_claim_severity:,.2f}")
        
        # Claim severity by province
        if 'Province' in df.columns:
            print("\nClaim Severity by Province:")
            province_severity = df_with_claims.groupby('Province')['TotalClaims'].mean().sort_values(ascending=False)
            print(province_severity.to_string())
        
        # Claim severity by vehicle type
        if 'VehicleType' in df.columns:
            print("\nClaim Severity by Vehicle Type (Top 10):")
            vehicle_severity = df_with_claims.groupby('VehicleType')['TotalClaims'].mean().sort_values(ascending=False).head(10)
            print(vehicle_severity.to_string())

## 8. Outlier Detection

In [ ]:
# Outlier detection using IQR method
def detect_outliers_iqr(data, column):
    """Detect outliers using the IQR method"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    return outliers, lower_bound, upper_bound

# Detect outliers in key columns
outlier_columns = ['TotalPremium', 'TotalClaims', 'SumInsured']

print("Outlier Detection (IQR Method)")
print("="*80)

for col in outlier_columns:
    if col in df.columns:
        outliers, lower, upper = detect_outliers_iqr(df, col)
        outlier_pct = (len(outliers) / len(df)) * 100
        
        print(f"\n{col}:")
        print(f"  Lower Bound: R {lower:,.2f}")
        print(f"  Upper Bound: R {upper:,.2f}")
        print(f"  Outliers: {len(outliers):,} ({outlier_pct:.2f}%)")
        
        if len(outliers) > 0:
            print(f"  Min Outlier: R {outliers[col].min():,.2f}")
            print(f"  Max Outlier: R {outliers[col].max():,.2f}")

In [ ]:
# Boxplot for TotalClaims by VehicleType
if 'VehicleType' in df.columns and 'TotalClaims' in df.columns:
    # Get top 10 vehicle types by count
    top_vehicle_types = df['VehicleType'].value_counts().head(10).index
    df_top_vehicles = df[df['VehicleType'].isin(top_vehicle_types)].copy()
    
    plt.figure(figsize=(14, 8))
    sns.boxplot(data=df_top_vehicles, y='VehicleType', x='TotalClaims', palette='Set2')
    plt.xlabel('Total Claims (R)', fontsize=12)
    plt.ylabel('Vehicle Type', fontsize=12)
    plt.title('Distribution of Total Claims by Vehicle Type (Top 10)\nOutliers Visible', 
              fontsize=14, fontweight='bold')
    plt.xlim(0, df_top_vehicles['TotalClaims'].quantile(0.95))  # Limit x-axis for better visibility
    plt.tight_layout()
    plt.savefig('../outputs/plots/boxplot_claims_by_vehicle.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Boxplot saved to outputs/plots/boxplot_claims_by_vehicle.png")

## 9. Time Series Analysis

In [ ]:
# Monthly trends
if 'TransactionMonth' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    monthly_data = df.groupby('TransactionMonth').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).reset_index()
    
    monthly_data.columns = ['Month', 'TotalPremium', 'TotalClaims', 'PolicyCount']
    monthly_data['LossRatio'] = (monthly_data['TotalClaims'] / monthly_data['TotalPremium']) * 100
    
    print("Monthly Trends:")
    print("="*80)
    print(monthly_data.to_string(index=False))
    
    # Visualize monthly trends
    fig, ax1 = plt.subplots(figsize=(14, 6))
    
    ax1.plot(monthly_data['Month'], monthly_data['TotalPremium'], 
             marker='o', linewidth=2, color='blue', label='Total Premium')
    ax1.plot(monthly_data['Month'], monthly_data['TotalClaims'], 
             marker='s', linewidth=2, color='red', label='Total Claims')
    ax1.set_xlabel('Month', fontsize=12)
    ax1.set_ylabel('Amount (R)', fontsize=12)
    ax1.tick_params(axis='y')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    plt.title('Monthly Trends: Premiums vs Claims\nFebruary 2014 - August 2015', 
              fontsize=14, fontweight='bold')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('../outputs/plots/monthly_trends.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\nMonthly trends plot saved to outputs/plots/monthly_trends.png")

## 10. Correlation Analysis

In [ ]:
# Select numeric columns for correlation
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Remove ID columns and temporal features
exclude_cols = ['UnderwrittenCoverID', 'PolicyID', 'TransactionYear', 'TransactionMonthNum', 'TransactionQuarter']
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

# Calculate correlation matrix
if len(numeric_cols) > 0:
    correlation_matrix = df[numeric_cols].corr()
    
    # Visualize correlation matrix
    plt.figure(figsize=(16, 14))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('Correlation Matrix of Numeric Variables', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('../outputs/plots/correlation_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Correlation matrix saved to outputs/plots/correlation_matrix.png")
    
    # Find strong correlations
    print("\nStrong Correlations (|r| > 0.7):")
    print("="*60)
    
    strong_corr = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > 0.7:
                strong_corr.append((
                    correlation_matrix.columns[i],
                    correlation_matrix.columns[j],
                    correlation_matrix.iloc[i, j]
                ))
    
    if strong_corr:
        for var1, var2, corr in sorted(strong_corr, key=lambda x: abs(x[2]), reverse=True):
            print(f"{var1} <-> {var2}: {corr:.3f}")
    else:
        print("No strong correlations found.")

## 11. Additional Visualizations

In [ ]:
# Premium vs Claims scatter plot
if 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    # Sample data for better visualization
    sample_size = min(10000, len(df))
    df_sample = df.sample(n=sample_size, random_state=42)
    
    plt.figure(figsize=(12, 8))
    plt.scatter(df_sample['TotalPremium'], df_sample['TotalClaims'], 
                alpha=0.5, s=20, c='steelblue', edgecolors='none')
    
    # Add diagonal line (break-even)
    max_val = max(df_sample['TotalPremium'].max(), df_sample['TotalClaims'].max())
    plt.plot([0, max_val], [0, max_val], 'r--', linewidth=2, label='Break-even line')
    
    plt.xlabel('Total Premium (R)', fontsize=12)
    plt.ylabel('Total Claims (R)', fontsize=12)
    plt.title(f'Premium vs Claims Relationship\n(Sample of {sample_size:,} policies)', 
              fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('../outputs/plots/premium_vs_claims_scatter.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Scatter plot saved to outputs/plots/premium_vs_claims_scatter.png")

In [ ]:
# Distribution of TotalPremium and TotalClaims
if 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # TotalPremium distribution
    axes[0].hist(df['TotalPremium'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0].set_xlabel('Total Premium (R)', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    axes[0].set_title('Distribution of Total Premium', fontsize=12, fontweight='bold')
    axes[0].axvline(df['TotalPremium'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    axes[0].axvline(df['TotalPremium'].median(), color='green', linestyle='--', linewidth=2, label='Median')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # TotalClaims distribution
    axes[1].hist(df['TotalClaims'], bins=50, color='coral', edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Total Claims (R)', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)
    axes[1].set_title('Distribution of Total Claims', fontsize=12, fontweight='bold')
    axes[1].axvline(df['TotalClaims'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    axes[1].axvline(df['TotalClaims'].median(), color='green', linestyle='--', linewidth=2, label='Median')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../outputs/plots/premium_claims_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("Distribution plots saved to outputs/plots/premium_claims_distributions.png")

## 12. Summary and Key Findings

In [ ]:
print("="*80)
print("EXPLORATORY DATA ANALYSIS - SUMMARY")
print("="*80)

print("\n1. DATASET OVERVIEW")
print(f"   - Total Records: {len(df):,}")
print(f"   - Total Columns: {len(df.columns)}")
print(f"   - Date Range: {df['TransactionMonth'].min()} to {df['TransactionMonth'].max()}")

print("\n2. FINANCIAL SUMMARY")
print(f"   - Total Premium Collected: R {df['TotalPremium'].sum():,.2f}")
print(f"   - Total Claims Paid: R {df['TotalClaims'].sum():,.2f}")
print(f"   - Overall Loss Ratio: {overall_loss_ratio:.2f}%")

print("\n3. DATA QUALITY")
print(f"   - Columns with Missing Values: {len(missing_data)}")
print(f"   - Duplicate Records: {df.duplicated().sum()}")

print("\n4. KEY INSIGHTS")
if 'Province' in df.columns:
    best_province = province_analysis.iloc[0]
    worst_province = province_analysis.iloc[-1]
    print(f"   - Best Province (Lowest Loss Ratio): {best_province['Province']} ({best_province['LossRatio']:.2f}%)")
    print(f"   - Worst Province (Highest Loss Ratio): {worst_province['Province']} ({worst_province['LossRatio']:.2f}%)")

print("\n5. VISUALIZATIONS CREATED")
print("   - Missing values analysis")
print("   - Loss ratio by province")
print("   - Loss ratio by vehicle type")
print("   - Loss ratio by gender")
print("   - Monthly trends (premiums vs claims)")
print("   - Boxplot of claims by vehicle type")
print("   - Correlation matrix")
print("   - Premium vs claims scatter plot")
print("   - Distribution plots")

print("\n6. NEXT STEPS")
print("   - Implement DVC for data version control (Task 2)")
print("   - Perform A/B hypothesis testing (Task 3)")
print("   - Build predictive models (Task 4)")

print("\n" + "="*80)
print("EDA COMPLETED SUCCESSFULLY!")
print("="*80)